In [1]:
%run load_base_runs.ipynb

In [2]:
import pandas
%load_ext autoreload

%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from mlflow import MlflowClient
import mlflow
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import collections
import matplotlib as mpl
import numpy as np
from collections import defaultdict
import scienceplots
from sklearn import datasets
from sklearn.datasets import make_blobs
from cycler import cycler
from pathlib import Path
plt.style.use(['science', 'bright'])


In [4]:
NOTE_PARAMS = ['metrics.no_of_predicates_median', 'metrics.complex_clfs',
                     'metrics.rules', 'metrics.no_of_predicates_max',
                     'metrics.dummy_clfs', 'metrics.no_of_predicates_min',
                     'metrics.all_clfs', 'params.min_split_percentage',
                    'params.complexity_measure',
                     'params.base_clf','params.min_samples']

In [5]:
NOTE_PARAMS = ['cls_data', 'params.subspaces', 'params.n_clf', "params.cv"]


In [6]:
if cached("24.07-note"):
    note_runs = load_cache("24.07-note")
else:
    note_runs = get_runs_for("24.07-note")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report")))\
        [DATASET_PARAMS + ['cls_report'] + NOTE_PARAMS]

KeyError: "['cls_data'] not in index"

In [ ]:
cache(note_runs, "24.07-note")

In [ ]:
note_runs_exploded = note_runs.explode('cls_report').rename(columns={"cls_report": "metric"}).assign(cls_report=note_runs['cls_report'].apply(lambda it: it.values()).explode()).reset_index(drop=True)

In [ ]:
note_runs_exploded = note_runs_exploded.assign(complexity_dts=lambda row: row['cls_report'].apply(calculate_rf_compexity))\
    .assign(complexity_rules=lambda row: row['cls_report'].apply(lambda it: sum(it['rules'])))\
    .assign(complexity=lambda row: row['complexity_dts'] + row['complexity_rules'])
    

In [ ]:
note_runs_exploded['imb_cls_report'] = note_runs_exploded['cls_report']

In [ ]:
note_runs_exploded = note_runs_exploded\
    .assign(**classification_metrics_extractors)\
    .assign(complexity_dts=lambda row: row['cls_report'].apply(calculate_rf_compexity))\
    .assign(complexity_rules=lambda row: row['cls_report'].apply(lambda it: sum(it['rules'])))\
    .assign(complexity=lambda row: row['complexity_dts'] + row['complexity_rules'])
    

In [ ]:
note_runs_exploded.groupby('metric')['test_balanced_accuracy'].mean()

In [ ]:

DATASET_PARAMS = ['params.should_take_test', 'params.dataset', 'params.data_shuffle_random_state']
RF_METRICS = ['metrics.depth_mean',
              'metrics.depth_min',
              'metrics.depth_median',
              'metrics.n_leaves_max',
              'metrics.n_leaves_min',
              'metrics.n_leaves_mean',
              'metrics.depth_max',
              'metrics.n_leaves_median',]


In [ ]:
def calculate_accuracy(row):
    return (row['metrics.tp'] + row['metrics.tn']) / (row['metrics.fp'] + row['metrics.tp'] + row['metrics.tn'] + row['metrics.fn'])

In [ ]:
def calculate_recall(row):
    return row['metrics.tp'] / (row['metrics.tp'] + row['metrics.fn'])

def calculate_precision(row):
    return row['metrics.tp'] / (row['metrics.tp'] + row['metrics.fp'])

In [ ]:
base_runs_dt = get_runs_for("base_runs_dt")[CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.depth', "metrics.n_leaves"]]\
    .assign(test_accuracy=calculate_accuracy) \
    .assign(test_recall=calculate_recall)\
    .assign(test_precision=calculate_precision)

In [ ]:
complex_dt.json

In [ ]:
base_runs_rf = get_runs_for("base_runs_rf_extended")\
    .assign(complex_dt_metrics=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "complex_dt.json")))\
    [CLASSIFICATION_METRICS + DATASET_PARAMS + RF_METRICS]\
    .assign(test_accuracy=calculate_accuracy) \
    .assign(test_recall=calculate_recall) \
    .assign(test_precision=calculate_precision)

In [ ]:
base_runs_rf['complex_dt_metrics']

In [ ]:
base_runs_oner = (get_runs_for("base_runs_oner")[CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.rules_no', 'metrics.depth']]

    .assign(test_accuracy=calculate_accuracy)) \
    .assign(test_recall=calculate_recall) \
    .assign(test_precision=calculate_precision)


In [ ]:
base_runs_rulefit = (get_runs_for("base_runs_rulefit")[CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.max_rules', 'metrics.rules_no', 'metrics.terms_no']]
    .assign(test_accuracy=calculate_accuracy)) \
    .assign(test_recall=calculate_recall) \
    .assign(test_precision=calculate_precision)


In [ ]:
base_runs_greedy = (get_runs_for("base_runs_greedy")[CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.depth', 'metrics.rules_no']]
        .assign(test_accuracy=calculate_accuracy))\
    .assign(test_recall=calculate_recall) \
    .assign(test_precision=calculate_precision)

In [ ]:
quad_split_runs_df =get_runs_for("06.07-quadsplit-3")[CLASSIFICATION_METRICS + DATASET_PARAMS + QUAD_SPLIT_PARAMS] \
    .assign(test_accuracy=calculate_accuracy) \
    .assign(test_recall=calculate_recall) \
    .assign(test_precision=calculate_precision)

In [ ]:
base_runs_rf.rename(columns={c: c+'_rf' for c in base_runs_rf.columns if 'metric' in c or 'test_' in c}, inplace=True)
base_runs_dt.rename(columns={c: c+'_dt' for c in base_runs_dt.columns if 'metric' in c  or 'test_' in c}, inplace=True)
base_runs_greedy.rename(columns={c: c+'_greedy' for c in base_runs_greedy.columns if 'metric' in c or 'test_' in c}, inplace=True)
base_runs_oner.rename(columns={c: c+'_oner' for c in base_runs_oner.columns if 'metric' in c or 'test_' in c}, inplace=True)
base_runs_rulefit.rename(columns={c: c+'_rulefit' for c in base_runs_rulefit.columns if 'metric' in c or 'test_' in c}, inplace=True)


In [ ]:
base_runs_together_df = pd.merge(
    base_runs_dt,
    base_runs_rf,
    how='right',
    on=DATASET_PARAMS,
    suffixes=('_dt', '_rf')
)\
    .merge(base_runs_greedy, how='right', on=DATASET_PARAMS, suffixes=('', '_greedy'))\
    .merge(base_runs_oner, how='right', on=DATASET_PARAMS, suffixes=('', '_oner'))\
    .merge(base_runs_rulefit, how='right', on=DATASET_PARAMS, suffixes=('', '_rulefit'))\
    .dropna()
note_runs_df = pd.merge(
    base_runs_together_df,
    note_runs,
    how='right',
    on=DATASET_PARAMS,
    suffixes=('', '_opt')
)
note_runs_df = pd.merge(
    note_runs_df,
    complexities_df,
    how='left',
    on=DATASET_PARAMS
)

In [ ]:
note_runs_df[['test_accuracy_rf', 'test_accuracy_dt', 'test_accuracy', 'test_accuracy_by_acc']].mean().plot(kind='bar')

In [ ]:
LABELS_THAT_SHOULD_BE =["better", "equal", "worse"] 

In [ ]:
quad_split_df.dropna(inplace=True)

In [ ]:
LIMITING_QUERY = "`params.min_samples` == '28' and `params.min_split_percentage` == '0.183'"

In [ ]:
quad_split_df.dropna(inplace=True)

In [ ]:
METRICS = ['test_accuracy']

In [ ]:
SUFFIXES = ["_dt", "_rf", "_oner", "_rulefit", "_greedy"]

In [ ]:
def define_comparison(row, suffix, metric):
    if row[f'{metric}_equal{suffix}']:
        return "equal"
    if row[f'{metric}_better{suffix}']:
        return "better"
    else:
        return "worse"

In [ ]:
for suffix in SUFFIXES:
    for metric in METRICS:
        note_runs_df[f'{metric}_diff_to{suffix}'] = note_runs_df[metric] - note_runs_df[f'{metric}{suffix}']
        note_runs_df[f'{metric}_better_equal{suffix}'] = note_runs_df[metric] >= note_runs_df[f'{metric}{suffix}']
        note_runs_df[f'{metric}_better{suffix}'] = note_runs_df[metric] > note_runs_df[f'{metric}{suffix}']
        note_runs_df[f'{metric}_worse{suffix}'] = ~note_runs_df[f'{metric}_better_equal{suffix}']
        note_runs_df[f'{metric}_worse_equal{suffix}'] = ~note_runs_df[f'{metric}_better{suffix}']
        note_runs_df[f'{metric}_equal{suffix}'] = note_runs_df[metric] == quad_split_df[f'{metric}{suffix}']
        note_runs_df[f'{metric}_comparison{suffix}'] = note_runs_df.apply(lambda row: define_comparison(row, suffix, metric), axis=1)

In [ ]:
# note_runs_df['complexity_rf'] = note_runs_df['metrics.depth_median_rf'] * 32 * note_runs_df['metrics.n_leaves_median_rf']
# note_runs_df['complexity'] =  note_runs_df['metrics.rules']
# note_runs_df['complexity_dt'] = note_runs_df['metrics.depth_dt'] * note_runs_df['metrics.n_leaves_dt']
# note_runs_df['complexity_rulefit'] = note_runs_df['metrics.rules_no_rulefit']
# note_runs_df['complexity_oner'] = note_runs_df['metrics.rules_no_oner']


In [ ]:
# for metric in COMPLEXITY_METRICS:
#     quad_split_df[f'{metric}-discretized'] = pd.qcut(quad_split_df[metric], 3, labels=["low", "medium", "high"])


In [ ]:
ALL_DISCRETIZED_METRICS = [f'{metric}-discretized' for metric in COMPLEXITY_METRICS]

In [ ]:
# compare_values = {
#     "low": 0,
#     "medium": 1,
#     "high": 2,
# }
# compare_values_inverted = {
#     v: k for k, v in compare_values.items()
# }
# def take_higher(row):
#     vals = [compare_values[it] for it in [row[0], row[1]]]
#     max_val = max(vals)
    
#     return compare_values_inverted[max_val]

In [ ]:
# quad_split_df['overall-complexity'] = quad_split_df[ALL_DISCRETIZED_METRICS].mode(axis=1).fillna('low').apply(take_higher, axis=1)

# COMPARISON TO ALGORITHSM

In [ ]:
labels = [f"test_accuracy_comparison{suffix}" for suffix in SUFFIXES]

note_runs_df[labels].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).plot(kind='bar')

In [ ]:
quad_split_df\
    .query(LIMITING_QUERY)\
    [ALL_DISCRETIZED_METRICS].apply(lambda x:x.value_counts())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

err_vars = quad_split_df.groupby(["params.complexity_measure"])[['test_accuracy', 'test_recall', 'test_precision']].var()

df_to_plot = quad_split_df.groupby(["params.complexity_measure"])[['test_accuracy', 'test_recall', 'test_precision']].mean()

df_to_plot.plot(kind='bar', ax=ax, yerr=err_vars)

ax.set_ylim(0.5, 1)

ax.legend(["accuracy", "precision", "recall"])
ax.set_xlabel("Complexity measure")
ax.set_ylabel("Metric value")
ax.axhline(df_to_plot.test_accuracy.max())
# for container in ax.containers:
#     ax.bar_label(container)
# handles, labels = ax.get_legend_handles_labels()
# fig.legend(handles, labels, loc='center right')
fig.savefig('../figures/quad/complexity-metric-comparison.pdf')

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(10, 5))

err_vars = quad_split_df.groupby(["params.base_clf"])[['test_accuracy', 'test_recall', 'test_precision']].var()
quad_split_df.groupby(["params.base_clf"])[['test_accuracy', 'test_recall', 'test_precision']].mean().plot(kind='bar', ax=ax, yerr=err_vars)

ax.set_ylim(0.5, 1)


ax.set_xlabel("Base classifier")
ax.set_ylabel("Metric value")
ax.legend(["accuracy", "precision", "recall"])


In [ ]:
LIMITING_QUERY = "`params.min_samples` == '28' and `params.min_split_percentage` == '0.183' and `params.base_clf` == 'dt' and `params.complexity_measure` == 'density'"

In [ ]:
def vals_to_percentage(l):
    total = sum(l)
    
    return [round(item/total*100,2) for item in l]
    

## WINS-LOSSES acc general

In [ ]:
def plot_bar(labels, values_dict, quants, ax):
    plot_labels = [l.split('_')[-1].upper() for l in labels]
    data = np.array(list(values_dict.values()))
    data_cum = data.cumsum(axis=1)
        
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, colname in enumerate(quants):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(plot_labels, widths, left=starts, height=0.5, label=colname)

        ax.bar_label(rects, label_type='center')




In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(10, 10))


ax1.set_title("Accuracy")
labels = [f"test_accuracy_comparison{suffix}" for suffix in SUFFIXES]
vals_to_plot = {k: vals_to_percentage(v.values()) for k,v in quad_split_df.query(LIMITING_QUERY)[labels].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).to_dict().items()}
plot_bar(labels, vals_to_plot, ['better', 'equal', 'worse'], ax1)

ax2.set_title("Recall")
labels = [f"test_recall_comparison{suffix}" for suffix in SUFFIXES]
vals_to_plot = {k: vals_to_percentage(v.values()) for k,v in quad_split_df.query(LIMITING_QUERY)[labels].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).to_dict().items()}
plot_bar(labels, vals_to_plot, ['better', 'equal', 'worse'], ax2)


ax3.set_title("Precision")
labels = [f"test_precision_comparison{suffix}" for suffix in SUFFIXES]
vals_to_plot = {k: vals_to_percentage(v.values()) for k,v in quad_split_df.query(LIMITING_QUERY)[labels].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).to_dict().items()}
plot_bar(labels, vals_to_plot, ['better', 'equal', 'worse'], ax3)

handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')

fig.savefig('../figures/quad/overall-performance-barplot.pdf')

In [ ]:
ACC_LABELS = [f"test_accuracy_comparison{suffix}" for suffix in SUFFIXES]
PRECISION_LABLES = [f"test_precision_comparison{suffix}" for suffix in SUFFIXES]
RECALL_LABLES = [f"test_recall_comparison{suffix}" for suffix in SUFFIXES]

In [ ]:
COMPLEXITIES_TO_CONSIDER = ['f2', 't4', 'c1', 'n3', 'l2', 'density']

In [ ]:
COMPLEXITIES_WITH_HIGHEST_VARIANCE = ['f2', 't4', 'c1', 'n3', 'l2', 'density']

In [ ]:
def make_sure_all_labels_are_there(df):
    missing_labels = set(LABELS_THAT_SHOULD_BE) - set(df.index)

    for label in missing_labels:
        df.loc[label] = pd.Series()

    return df.reindex(LABELS_THAT_SHOULD_BE)

In [ ]:
fig, axes = plt.subplots(3,3, sharey=True, figsize=(20, 10))

axes = axes.reshape(-1)

axes_iter = iter(axes)
plot_labels = [l.split('_')[-1].upper() for l in ACC_LABELS]



for labels, label in zip([ACC_LABELS, RECALL_LABLES, PRECISION_LABLES], ["Accuracy", "Recall", "Precision"]):
    for value in ['low', 'medium', 'high']:
        ax = next(axes_iter)
        vals_to_plot = {k: vals_to_percentage(v.values()) for k,v in make_sure_all_labels_are_there(quad_split_df.query(LIMITING_QUERY).query(f"`overall-complexity` == '{value}'")[labels].apply(pd.Series.value_counts)).fillna(0).to_dict().items()}
    
        data = np.array(list(vals_to_plot.values()))
        data_cum = data.cumsum(axis=1)
        ax.invert_yaxis()
        ax.xaxis.set_visible(False)
        # ax.yaxis.set_visible(False)
        ax.set_xlim(0, np.sum(data, axis=1).max())

        for i, colname in enumerate(['better', 'equal', 'worse']):
            widths = data[:, i]
            starts = data_cum[:, i] - widths
            rects = ax.barh(plot_labels, widths, left=starts, height=0.5, label=colname)
            ax.bar_label(rects, label_type='center')
    
        ax.set_title(f"complexity {value} {label}")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')
fig.savefig("../figures/quad/overall-complexity-barplot.pdf")

In [ ]:
fig, axes = plt.subplots(1,3, sharey=True, figsize=(20, 5))

axes = axes.reshape(-1)

axes_iter = iter(axes)
plot_labels = [l.split('_')[-1].upper() for l in ACC_LABELS]


for value in ['low', 'medium', 'high']:

    ax = next(axes_iter)
    vals_to_plot = {k:  vals_to_percentage(v.values()) for k,v in quad_split_df.query(LIMITING_QUERY).query(f"`overall-complexity` == '{value}'")[ACC_LABELS].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).fillna(0).to_dict().items()}

    data = np.array(list(vals_to_plot.values()))
    data_cum = data.cumsum(axis=1)
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    # ax.yaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, colname in enumerate(['better', 'equal', 'worse']):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(plot_labels, widths, left=starts, height=0.5, label=colname)
        ax.bar_label(rects, label_type='center')

    ax.set_title(f"complexity {value}")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')
fig.savefig("../figures/quad/accuracy-overall-complexity-barplot.pdf")

## Models complexity

In [ ]:
MODEL_COMPLEXITY_LABELS = ['complexity',"complexity_rf", "complexity_dt", 'complexity_rulefit', "complexity_oner"]
MODEL_COMPLEXITY_LABELS_WO_RF = ['complexity', "complexity_dt", 'complexity_rulefit', "complexity_oner"]


In [ ]:
quad_split_df[MODEL_COMPLEXITY_LABELS]

In [ ]:
quad_split_df.query(LIMITING_QUERY).groupby("overall-complexity")[MODEL_COMPLEXITY_LABELS_WO_RF].mean().plot(kind='bar')

In [ ]:
quad_split_df.query(LIMITING_QUERY).groupby("overall-complexity")[MODEL_COMPLEXITY_LABELS_WO_RF].var().plot(kind='bar')

In [ ]:
order = pd.Categorical([ 'low', 'medium', 'high'],ordered=True)

In [ ]:
quad_split_df \
    .query(LIMITING_QUERY) \
    .groupby('overall-complexity')[['complexity_dt', 'complexity']] \
    .mean().transpose()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15, 7))

quad_split_df\
    .query(LIMITING_QUERY) \
    [['test_accuracy_better_rf'] + ALL_DISCRETIZED_METRICS] \
    .melt('test_accuracy_better_rf')\
    .groupby(['variable', 'value'], as_index=False)['test_accuracy_better_rf'].var()\
    .groupby('variable')['test_accuracy_better_rf'].var()\
    .rename(index=lambda name: name.split('-')[0]).plot(kind='pie', ax=ax1)

ax1.set_title("Variance of wins against Random Forest")

quad_split_df\
    .query(LIMITING_QUERY) \
    [['test_accuracy_better_dt'] + ALL_DISCRETIZED_METRICS] \
    .melt('test_accuracy_better_dt')\
    .groupby(['variable', 'value'], as_index=False).apply(lambda x: np.sum(x)/len(x))\
    .groupby('variable')['test_accuracy_better_dt'].var()\
    .rename(index=lambda name: name.split('-')[0]).plot(kind='pie', ax=ax2)

ax2.set_title("Variance of wins against Decision Tree")
fig.savefig('../figures/quad/variance-of-wins.pdf')

In [ ]:
most_variance_metrics = quad_split_df\
    .query(LIMITING_QUERY) \
    [['test_accuracy_better_dt'] + ALL_DISCRETIZED_METRICS] \
    .melt('test_accuracy_better_dt')\
    .groupby(['variable', 'value'], as_index=False).apply(lambda x: np.sum(x)/len(x))\
    .groupby('variable')['test_accuracy_better_dt'].var().sort_values(ascending=False)[:5].index

In [ ]:
most_variance_metrics = set(most_variance_metrics).union(quad_split_df\
    .query(LIMITING_QUERY) \
    [['test_accuracy_better_rf'] + ALL_DISCRETIZED_METRICS] \
    .melt('test_accuracy_better_rf')\
    .groupby(['variable', 'value'], as_index=False).apply(lambda x: np.sum(x)/len(x))\
    .groupby('variable')['test_accuracy_better_rf'].var().sort_values(ascending=False)[:4].index)

In [ ]:
COMPLEXITIES_TO_CONSIDER = [m.split("-")[0]  for m in most_variance_metrics]

In [ ]:
COMPLEXITIES_TO_CONSIDER

In [ ]:
fig, axes = plt.subplots(len(COMPLEXITIES_TO_CONSIDER),3,figsize=(20, 20), sharey='row')

axes = axes.reshape(-1)

axes_iter = iter(axes)
for complexity in COMPLEXITIES_TO_CONSIDER:
    plot_labels = [l.split('_')[-1].upper() for l in ACC_LABELS]
    
    
    for value in ['low', 'medium', 'high']:

        ax = next(axes_iter)
        vals_to_plot = {k:  vals_to_percentage(v.values()) for k,v in quad_split_df.query(LIMITING_QUERY).query('`params.base_clf` == "dt"').query(f"`{complexity}-discretized` == '{value}'")[ACC_LABELS].apply(pd.Series.value_counts).reindex(LABELS_THAT_SHOULD_BE).fillna(0).to_dict().items()}

        data = np.array(list(vals_to_plot.values()))
        data_cum = data.cumsum(axis=1)
        ax.invert_yaxis()
        ax.xaxis.set_visible(False)
        ax.set_xlim(0, np.sum(data, axis=1).max())
    
        for i, colname in enumerate(['better', 'equal', 'worse']):
            widths = data[:, i]
            starts = data_cum[:, i] - widths
            rects = ax.barh(plot_labels, widths, left=starts, height=0.5, label=colname)
            ax.bar_label(rects, label_type='center')
    
        ax.set_title(f"{complexity}-{value}")


handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')
fig.savefig('../figures/quad/accuracy-with-variance-complexity-metrics-barplot.pdf')
